In [ ]:
dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [ ]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

### Testes Modulares - Referente ao tópico 3/4 - 3.6.1 / 4.4.1 Limite Final

### Geração de Massa

In [ ]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({'solicitante.rendaLiquidaPicpay':[100],
                             'payloadHomol.intermediarias.alavancagem' :[1],
                             'payloadHomol.intermediarias.limitePiso':[100],
                             'payloadHomol.intermediarias.limiteTeto':[40000],
                             'payloadHomol.intermediarias.limiteFixo':[0]
                            })

In [ ]:
#Criação das massas com casos forçados
#Digito CPF Concomitante CP
massa_limiteFinal = sherlock.Gera_Massa(base_entrada)
massa_limiteFinal.gera_cenario('solicitante.rendaLiquidaPicpay',[-99999,100,1000,10000,3733,7311,983.20])
massa_limiteFinal.gera_cenario('payloadHomol.intermediarias.alavancagem',[0.2,0.5,0.6,0.8,1])
massa_limiteFinal.gera_cenario('payloadHomol.intermediarias.limitePiso',[100,200,300,1000,4000,4500,6000,7500,10000])
massa_limiteFinal.gera_cenario('payloadHomol.intermediarias.limiteFixo',[0,100,200])
massa_limiteFinal.gera_cenario('payloadHomol.intermediarias.limiteTeto',[100,200,700,1100,2000,2300,2400,2500,3000,3200,3400,4000,4500,5000,5500,6000,7000,7500,10000,12000,15000,20000,30000,40000])

massa_limiteFinal.set_bom_arqv()
massa_limiteFinal.converte_massa()

In [ ]:
massa_limiteFinal.get_massa_final()

### Execução Modular e Geração de CSV

In [ ]:
listaResultado = rp.execucaoModular(massa_limiteFinal.get_massa_json(),rp.LimiteFinal)

##### Valida se todas condições estão sendo abordadas

In [ ]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['saidas']['limiteFinal'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

In [ ]:
# Renomeação das colunas para o cabeçalho do RMA
massa_limiteFinal.rename_cabecalho()

In [ ]:
massa_limiteFinal.atribui_expected(listaResultado,"payloadHomol.saidas.limiteFinal","limite")
massa_limiteFinal.get_massa_dm()

### Conversão da massa para CSV

In [ ]:
massa_limiteFinal.get_massa_dm().to_csv('C:/Users/T51657/Desktop/Homologação/Modelos Online/v2/massas/Modulares/massaLimiteFinal.csv',index=False)